In [3]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from Postgres_Login import username
from Postgres_Login import password

In [4]:
#Store CSV into DataFrame
csv_file = "Resources/netflix_dataset.csv"
netflix_data_df = pd.read_csv(csv_file)
netflix_data_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [5]:
#Drop 'Movies', only want 'TV Shows'
new_netflix_data_df = netflix_data_df.loc[netflix_data_df['type'] == 'TV Show']
new_netflix_data_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
5,80163890,TV Show,Apaches,NaN,"Alberto Ammann, Eloy Azorín, Verónica Echegui,...",Spain,"September 8, 2017",2016,TV-MA,1 Season,"Crime TV Shows, International TV Shows, Spanis...",A young journalist is forced into a life of cr...
8,80117902,TV Show,Fire Chasers,NaN,NaN,United States,"September 8, 2017",2017,TV-MA,1 Season,"Docuseries, Science & Nature TV","As California's 2016 fire season rages, brave ..."
26,80244601,TV Show,Castle of Stars,NaN,"Chaiyapol Pupart, Jintanutda Lummakanon, Worra...",NaN,"September 7, 2018",2015,TV-14,1 Season,"International TV Shows, Romantic TV Shows, TV ...",As four couples with different lifestyles go t...


In [6]:
#Create new data with select columns (Title, Genres, No. of Seasons, TV Rating) 
#Add new column and name it 'Streaming Service'
clean_netflix_data_df = new_netflix_data_df[['title', 'listed_in', 'duration', 'rating', 'release_year']]
final_netflix_data = clean_netflix_data_df.rename(columns = {"title" : "Title of Show",
                                                          "listed_in" : "Genres",
                                                          "duration" : "No. of Seasons",
                                                          "rating" : "TV Rating"})
final_netflix_data['Streaming Service'] = 'yes'
final_netflix_data.head()

,Title of Show,Genres,No. of Seasons,TV Rating,release_year,Streaming Service
2,Transformers Prime,Kids' TV,1 Season,TV-Y7-FV,2013,yes
3,Transformers: Robots in Disguise,Kids' TV,1 Season,TV-Y7,2016,yes
5,Apaches,"Crime TV Shows, International TV Shows, Spanis...",1 Season,TV-MA,2016,yes
8,Fire Chasers,"Docuseries, Science & Nature TV",1 Season,TV-MA,2017,yes
26,Castle of Stars,"International TV Shows, Romantic TV Shows, TV ...",1 Season,TV-14,2015,yes


In [11]:
#Connect to local database
rds_connection_string = f"{username}:{password}@localhost:5432/streaming_services"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
#Use pandas to load csv converted DataFrame into database
final_netflix_data.to_sql(name='netflix2', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from netflix2', con=engine).head()

,Title of Show,Genres,No. of Seasons,TV Rating,release_year,Streaming Service
0,Transformers Prime,Kids' TV,1 Season,TV-Y7-FV,2013,yes
1,Transformers: Robots in Disguise,Kids' TV,1 Season,TV-Y7,2016,yes
2,Apaches,"Crime TV Shows, International TV Shows, Spanis...",1 Season,TV-MA,2016,yes
3,Fire Chasers,"Docuseries, Science & Nature TV",1 Season,TV-MA,2017,yes
4,Castle of Stars,"International TV Shows, Romantic TV Shows, TV ...",1 Season,TV-14,2015,yes
